In [53]:
from utils import *
import warnings
warnings.filterwarnings('ignore')

### FNDDS data

In [54]:
# The most recent three years of FNDDS tables contain nutrition data. The column names are slightly different.
df_1516 = pd.read_excel('../data/2015-2016 Ingredients.xlsx', skiprows=1)
df_1718 = pd.read_excel('../data/2017-2018 Ingredients.xlsx', skiprows=1)
df_1920 = pd.read_excel('../data/2019-2020 Ingredients.xlsx', skiprows=1)

# Unify the column names.
df_1516 = df_1516.rename(columns={'WWEIA Category code': 'WWEIA Category number'})

# A small proportion of FNDDS data, such as code and descriptions change over the years.
# Here we take the latest version of data if there are duplicates.
df_fndds = pd.concat([df_1516, df_1718, df_1920])
df_fndds = df_fndds[['Food code', 'Main food description', 'WWEIA Category number', 'WWEIA Category description', 'Ingredient code', 'Ingredient description']]
df_fndds = df_fndds.drop_duplicates(subset=['Food code', 'WWEIA Category number', 'Ingredient code'], keep='last')
df_fndds = df_fndds.sort_values(by='Food code')

# This table records the connections between food and ingredients.
df_fndds = df_fndds.rename(columns={'Food code': 'food_id', 'Main food description': 'food_desc', 'WWEIA Category number': 'WWEIA_id',
                        'WWEIA Category description': 'WWEIA_desc', 'Ingredient code': 'ingredient_id', 'Ingredient description': 'ingredient_desc'})

In [55]:
# There are 9260 foods in total.
# This is the FNDDS dataset. 
print(len(set(df_fndds['food_id'].tolist())))
df_fndds.head()

9260


,food_id,food_desc,WWEIA_id,WWEIA_desc,ingredient_id,ingredient_desc
0,11000000,"Milk, human",9602,Human milk,1107,"Milk, human, mature, fluid (For Reference Only)"
4,11100000,"Milk, NFS",1004,"Milk, reduced fat",1085,"Milk, nonfat, fluid, with added vitamin A and ..."
3,11100000,"Milk, NFS",1004,"Milk, reduced fat",1082,"Milk, lowfat, fluid, 1% milkfat, with added vi..."
2,11100000,"Milk, NFS",1004,"Milk, reduced fat",1079,"Milk, reduced fat, fluid, 2% milkfat, with add..."
1,11100000,"Milk, NFS",1004,"Milk, reduced fat",1077,"Milk, whole, 3.25% milkfat, with added vitamin D"


In [69]:
df_fndds.to_csv('../processed_data/fndds.csv', index=False)

### Dietary Record Data

In [56]:
years = ['0304', '0506', '0708', '0910', '1112', '1314', '1516', '1718', '1720']
year_char = 'C'
type_dietary = 'dietary'
df_IFF1 = concat_data_across_years(type_dietary, 'DR1IFF', years, year_char)
df_IFF2 = concat_data_across_years(type_dietary, 'DR2IFF', years, year_char)

# Food and nutrition data
food_columns_1 = ['SEQN', 'food_id', 'DR1IGRMS',
 'DR1IKCAL', 'DR1IPROT', 'DR1ICARB', 'DR1ISUGR', 'DR1IFIBE', 'DR1ITFAT',
 'DR1ISFAT', 'DR1IMFAT', 'DR1IPFAT', 'DR1ICHOL', 'DR1IATOC', 'DR1IATOA',
 'DR1IRET', 'DR1IVARA', 'DR1IACAR', 'DR1IBCAR', 'DR1ICRYP', 'DR1ILYCO',
 'DR1ILZ', 'DR1IVB1', 'DR1IVB2', 'DR1INIAC', 'DR1IVB6', 'DR1IFOLA',
 'DR1IFA', 'DR1IFF', 'DR1IFDFE', 'DR1ICHL', 'DR1IVB12', 'DR1IB12A',
 'DR1IVC', 'DR1IVD', 'DR1IVK', 'DR1ICALC', 'DR1IPHOS', 'DR1IMAGN',
 'DR1IIRON', 'DR1IZINC', 'DR1ICOPP', 'DR1ISODI', 'DR1IPOTA', 'DR1ISELE',
 'DR1ICAFF', 'DR1ITHEO', 'DR1IALCO', 'DR1IMOIS'
]
food_columns_2 = ['SEQN', 'food_id', 'DR2IGRMS',
 'DR2IKCAL', 'DR2IPROT', 'DR2ICARB', 'DR2ISUGR', 'DR2IFIBE', 'DR2ITFAT',
 'DR2ISFAT', 'DR2IMFAT', 'DR2IPFAT', 'DR2ICHOL', 'DR2IATOC', 'DR2IATOA',
 'DR2IRET', 'DR2IVARA', 'DR2IACAR', 'DR2IBCAR', 'DR2ICRYP', 'DR2ILYCO',
 'DR2ILZ', 'DR2IVB1', 'DR2IVB2', 'DR2INIAC', 'DR2IVB6', 'DR2IFOLA',
 'DR2IFA', 'DR2IFF', 'DR2IFDFE', 'DR2ICHL', 'DR2IVB12', 'DR2IB12A',
 'DR2IVC', 'DR2IVD', 'DR2IVK', 'DR2ICALC', 'DR2IPHOS', 'DR2IMAGN',
 'DR2IIRON', 'DR2IZINC', 'DR2ICOPP', 'DR2ISODI', 'DR2IPOTA', 'DR2ISELE',
 'DR2ICAFF', 'DR2ITHEO', 'DR2IALCO', 'DR2IMOIS'
]

df_IFF1 = df_IFF1.rename(columns={'DR1IFDCD': 'food_id'})
df_IFF1 = df_IFF1[food_columns_1].astype(float)
df_IFF2 = df_IFF2.rename(columns={'DR2IFDCD': 'food_id'})
df_IFF2 = df_IFF2[food_columns_2].astype(float)
df_food  = pd.DataFrame(np.vstack((df_IFF1.to_numpy(), df_IFF2.to_numpy())), columns=df_IFF1.columns)

df_IFF1 = df_IFF1[['SEQN', 'food_id']].astype(int).astype(str)
df_IFF2 = df_IFF2[['SEQN', 'food_id']].astype(int).astype(str)
df_IFF1['food_id'] = df_IFF1['food_id'].str.zfill(10)
df_IFF2['food_id'] = df_IFF2['food_id'].str.zfill(10)
df_food_user = pd.concat([df_IFF1, df_IFF2])

In [57]:
# This is the crosswalk between users and food records.
df_food_user

,SEQN,food_id
0,21005,0091745020
1,21005,0092410710
2,21005,0071201010
3,21005,0025230230
4,21005,0051301010
...,...,...
149490,124820,0027214100
149491,124820,0025210210
149492,124820,0071200100
149493,124820,0064104030


In [68]:
df_food_user.to_csv('../processed_data/food_user.csv', index=False)

In [58]:
# Create a new DataFrame for the nutritional data
df_nutrition = pd.DataFrame()
df_nutrition['food_id'] = df_food['food_id'].unique()
df_food = df_food.dropna(subset=['DR1IGRMS'])
for col in df_food.columns.tolist()[3:]:
    df_food[col] = df_food[col] / df_food['DR1IGRMS'] * 100

df_food.drop(['SEQN', 'DR1IGRMS'], axis=1, inplace=True)

df_food = df_food.groupby('food_id').mean().reset_index()
df_food = df_food.fillna(0)
df_food['food_id'] = df_food['food_id'].astype(int)
df_nutrition = df_nutrition.merge(df_food, how='left', on='food_id')

In [59]:
df_nutrition.head()

,food_id,DR1IKCAL,DR1IPROT,DR1ICARB,DR1ISUGR,DR1IFIBE,DR1ITFAT,DR1ISFAT,DR1IMFAT,DR1IPFAT,...,DR1IIRON,DR1IZINC,DR1ICOPP,DR1ISODI,DR1IPOTA,DR1ISELE,DR1ICAFF,DR1ITHEO,DR1IALCO,DR1IMOIS
0,91745020.0,395.278311,0.000000,98.001075,62.916355,0.000000,0.185781,0.000000,0.000000,0.000000,...,0.306136,0.001037,0.028588,36.937549,3.733017,0.415312,0.0,0.0,0.0,1.307854
1,92410710.0,41.036190,0.000000,10.600139,10.600139,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.050206,0.070027,0.006991,12.959341,0.997621,0.100043,0.0,0.0,0.0,89.300262
2,71201010.0,545.736044,6.562972,50.063185,1.741627,4.383135,37.155633,8.901914,11.665290,13.304546,...,1.613473,2.394710,0.397999,525.080821,1642.578659,8.109794,0.0,0.0,0.0,2.283774
3,25230230.0,109.660947,17.589882,1.129724,0.154148,0.000000,3.054334,0.826198,1.196596,0.306546,...,0.670906,1.745734,0.249023,1160.934459,532.559195,27.556117,0.0,0.0,0.0,74.363413
4,51301010.0,268.153759,10.461369,48.159033,5.827805,3.978369,3.783562,0.789484,0.818855,1.463109,...,3.492356,1.143217,0.163818,509.969638,175.926071,29.068944,0.0,0.0,0.0,35.407697


In [60]:
"""
This is merely for information. Not used in the pipeline.

We use the food code NHANES provided, which is more complete than FNDDS. For duplications, we also keep the latest records.
In this way, every food users reported has its corresponding food description.
We use this as the connections between users and food.
"""

food_dictionary = concat_data_across_years(type_dietary, 'DRXFCD', years, year_char)
food_dictionary = food_dictionary.rename(columns={'DRXFDCD': 'food_id', 'DRXFCLD': 'food_desc'})

food_dictionary = food_dictionary[['food_id', 'food_desc', 'years']]
food_dictionary['food_id'] = food_dictionary['food_id'].astype(int)
food_dictionary = food_dictionary.drop_duplicates(subset='food_id', keep='last')

food_nhanes_have = set(food_dictionary['food_id'].tolist())
len(food_nhanes_have)

11338

In [1]:
# By now, we have three tables: 
# `df_fndds` is the table for connecting foods to ingredients and categories; 
# `df_food_user` is the table for connecting foods to users who consume them; 
# `df_nutrition` is the table for the foods and their nutritions per 100g.

In [61]:
df_nutrition['food_id'] = df_nutrition['food_id'].astype(int).astype(str).str.zfill(10)
df_nutrition = df_nutrition.set_index('food_id')
df_nutrtion = df_nutrition.round(2)

In [62]:
df_nutrition.describe().round(2)

,DR1IKCAL,DR1IPROT,DR1ICARB,DR1ISUGR,DR1IFIBE,DR1ITFAT,DR1ISFAT,DR1IMFAT,DR1IPFAT,DR1ICHOL,...,DR1IIRON,DR1IZINC,DR1ICOPP,DR1ISODI,DR1IPOTA,DR1ISELE,DR1ICAFF,DR1ITHEO,DR1IALCO,DR1IMOIS
count,9639.00,9639.00,9639.00,9639.00,9639.00,9639.00,9639.00,9639.00,9639.00,9639.00,...,9639.00,9639.00,9639.00,9639.00,9639.00,9639.00,9639.00,9639.00,9639.00,9639.00
mean,192.52,8.12,21.37,7.79,1.87,8.52,2.65,3.12,2.03,29.86,...,1.84,1.31,0.14,338.78,216.74,11.75,2.47,5.28,0.13,60.21
std,139.39,8.02,22.09,13.03,2.77,10.66,3.85,4.54,3.65,67.46,...,3.89,3.06,0.36,417.81,204.00,24.16,72.69,37.48,1.55,27.08
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,77.92,2.05,6.21,0.94,0.21,1.94,0.44,0.50,0.32,0.00,...,0.47,0.33,0.05,135.81,117.61,1.23,0.00,0.00,0.00,46.56
50%,162.24,5.63,13.95,2.64,1.19,5.01,1.40,1.69,1.03,4.78,...,1.07,0.68,0.08,294.58,185.66,6.64,0.00,0.00,0.00,66.90
75%,272.69,11.60,26.10,7.62,2.38,12.00,3.53,4.34,2.40,38.42,...,1.87,1.38,0.13,453.61,262.68,18.66,0.00,0.00,0.00,81.81
max,901.29,78.13,100.00,100.00,77.26,100.00,82.87,75.22,66.85,3074.13,...,64.18,98.86,14.47,19645.56,6022.51,1917.13,4491.99,2050.64,36.66,99.98


### Tagging the food items

In [63]:
nutrition_mapping = {'DR1IKCAL': 'calorie', 'DR1IPROT': 'protein', 'DR1ICARB': 'carb', 'DR1ISUGR': 'sugar', 'DR1IFIBE': 'fiber', 
                     'DR1ISFAT': 'saturated_fat', 'DR1ICHOL': 'cholesterol', 'DR1ISODI': 'sodium', 'DR1ICALC': 'calcium', 'DR1IPHOS': 'phosphorus',
                     'DR1IPOTA': 'potassium', 'DR1IIRON': 'iron', 'DR1IFA': 'folic_acid', 'DR1IVC': 'vitamin_c', 'DR1IVD': 'vitamin_d', 'DR1IVB12': 'vitamin_b12'
                     }
nutrition_columns = ['DR1IKCAL', 'DR1IPROT', 'DR1ICARB', 'DR1ISUGR', 'DR1IFIBE', 'DR1ISFAT', 'DR1ICHOL', 
                    'DR1ISODI', 'DR1ICALC', 'DR1IPHOS', 'DR1IPOTA', 'DR1IIRON', 'DR1IFA', 'DR1IVC', 'DR1IVD', 'DR1IVB12']

In [64]:
thresholds = {
    'calorie': {'low': 40, 'high': 225},
    'protein': {'low': 10, 'high': 15},
    'carb': {'low': 55, 'high': 75},
    'sugar': {'low': 5, 'high': 22.5},
    'fiber': {'low': 3, 'high': 6},
    'saturated_fat': {'low': 1.5, 'high': 5},
    'cholesterol': {'low': 20, 'high': 40},
    'sodium': {'low': 120, 'high': 200},
    'calcium': {'low': 0, 'high': 150},
    'phosphorus': {'low': 0, 'high': 105},
    'potassium': {'low': 0, 'high': 525},
    'iron': {'low': 0, 'high': 3.3},
    'folic_acid': {'low': 0, 'high': 60},
    'vitamin_c': {'low': 0, 'high': 15},
    'vitamin_d': {'low': 0, 'high': 2.25},
    'vitamin_b12': {'low': 0, 'high': 0.36},
}

In [65]:
df_nutrition = df_nutrition[nutrition_columns]
df_nutrition = df_nutrition.rename(columns=nutrition_mapping)
for nutrient, cols in nutrition_mapping.items():
    low_col = f'low_{cols}'
    high_col = f'high_{cols}'
    
    df_nutrition[low_col] = df_nutrition[cols].apply(lambda x: 1 if x <= thresholds[cols]['low'] else 0)
    df_nutrition[high_col] = df_nutrition[cols].apply(lambda x: 1 if x > thresholds[cols]['high'] else 0)

In [66]:
# Have a look at the data
df_nutrition.describe().T

,count,mean,std,min,25%,50%,75%,max
calorie,9639.0,192.516941,139.394545,0.0,77.916779,162.235930,272.686853,901.287554
protein,9639.0,8.121850,8.022546,0.0,2.050089,5.630233,11.602435,78.132389
carb,9639.0,21.368806,22.089875,0.0,6.212551,13.948457,26.104188,100.000000
sugar,9639.0,7.789790,13.031943,0.0,0.936700,2.638770,7.615897,100.000000
fiber,9639.0,1.866523,2.768045,0.0,0.214974,1.191127,2.375233,77.262443
saturated_fat,9639.0,2.651805,3.845347,0.0,0.442835,1.398990,3.529069,82.865372
cholesterol,9639.0,29.863797,67.463802,0.0,0.000000,4.780511,38.420255,3074.130506
sodium,9639.0,338.780392,417.813502,0.0,135.810877,294.582629,453.609211,19645.555556
calcium,9639.0,73.661708,125.405424,0.0,14.830429,36.502458,92.839061,3433.481160
phosphorus,9639.0,126.676617,124.631671,0.0,43.016101,98.367389,177.459677,1676.785714


In [70]:
df_nutrition.to_csv('../processed_data/food_tagging.csv')